In [7]:
%matplotlib inline
!nvidia-smi

Tue Nov 14 01:02:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [3]:
!git clone https://github.com/VikashKumarShaw1994/DNN_invariant_time-rescaling.git DNN_invariant_sithcon

fatal: destination path 'DNN_invariant_sithcon' already exists and is not an empty directory.


In [5]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
#from deepsith import DeepSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

<class 'torch.cuda.FloatTensor'>


In [2]:
Braille_DICT = {
    ',': [
        [' ', '●'],
        [' ', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '.': [
        [' ', '●'],
        [' ', ' '],
        ['●', '●'],
        [' ', ' ']
    ],
    '(': [
        [' ', '●'],
        [' ', ' '],
        ['●', ' '],
        [' ', ' ']
    ],
    '-': [
        [' ', '●'],
        ['●', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '/': [
        [' ', ' '],
        ['●', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '?': [
        [' ', '●'],
        ['●', ' '],
        [' ', '●'],
        [' ', ' ']
    ],
    ')': [
        [' ', '●'],
        ['●', ' '],
        ['●', ' '],
        [' ', ' ']
    ],
    'A': [
        ['●', ' '],
        [' ', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'B': [
        ['●', '●'],
        [' ', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'C': [
        ['●', ' '],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'D': [
        ['●', ' '],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'E': [
        ['●', ' '],
        [' ', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'F': [
        ['●', '●'],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'G': [
        ['●', '●'],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'H': [
        ['●', '●'],
        [' ', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'I': [
        [' ', '●'],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'J': [
        [' ', '●'],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'K': [
        ['●', ' '],
        ['●', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'L': [
        ['●', '●'],
        ['●', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'M': [
        ['●', ' '],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'N': [
        ['●', ' '],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'O': [
        ['●', ' '],
        ['●', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'P': [
        ['●', '●'],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'Q': [
        ['●', '●'],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'R': [
        ['●', '●'],
        ['●', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'S': [
        [' ', '●'],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'T': [
        [' ', '●'],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'U': [
        ['●', ' '],
        ['●', ' '],
        [' ', '●'],
        ['●', ' ']
    ],
    'V': [
        ['●', '●'],
        ['●', ' '],
        [' ', '●'],
        ['●', ' ']
    ],
    'W': [
        [' ', '●'],
        [' ', '●'],
        ['●', '●'],
        ['●', ' ']
    ],
    'X': [
        ['●', ' '],
        ['●', '●'],
        [' ', '●'],
        ['●', ' ']
    ],
    'Y': [
        ['●', ' '],
        ['●', '●'],
        ['●', '●'],
        ['●', ' ']
    ],
    'Z': [
        ['●', ' '],
        ['●', ' '],
        ['●', '●'],
        ['●', ' ']
    ],
    '1': [
        ['●', ' '],
        [' ', ' '],
        [' ', ' '],
        [' ', '●']
    ],
    '2': [
        ['●', '●'],
        [' ', ' '],
        [' ', ' '],
        [' ', '●']
    ],
    '3': [
        ['●', ' '],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '4': [
        ['●', ' '],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ],
    '5': [
        ['●', ' '],
        [' ', ' '],
        ['●', ' '],
        [' ', '●']
    ],
    '6': [
        ['●', '●'],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '7': [
        ['●', '●'],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ],
    '8': [
        ['●', '●'],
        [' ', ' '],
        ['●', ' '],
        [' ', '●']
    ],
    '9': [
        [' ', '●'],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '0': [
        [' ', '●'],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ]
}


In [3]:
def braille_to_binary(braille_matrix):
    braille_code = ""
    for row in range(4):
        for col in range(2):
            braille_code += '1' if braille_matrix[row][col] == '●' else '0'

    return braille_code

Braille_CODE_DICT={}
for brk,brv in Braille_DICT.items():
    Braille_CODE_DICT[brk]=braille_to_binary(brv)
print(Braille_CODE_DICT)

{',': '01000000', '.': '01001100', '(': '01001000', '-': '01100000', '/': '00100000', '?': '01100100', ')': '01101000', 'A': '10000010', 'B': '11000010', 'C': '10010010', 'D': '10011010', 'E': '10001010', 'F': '11010010', 'G': '11011010', 'H': '11001010', 'I': '01010010', 'J': '01011010', 'K': '10100010', 'L': '11100010', 'M': '10110010', 'N': '10111010', 'O': '10101010', 'P': '11110010', 'Q': '11111010', 'R': '11101010', 'S': '01110010', 'T': '01111010', 'U': '10100110', 'V': '11100110', 'W': '01011110', 'X': '10110110', 'Y': '10111110', 'Z': '10101110', '1': '10000001', '2': '11000001', '3': '10010001', '4': '10011001', '5': '10001001', '6': '11010001', '7': '11011001', '8': '11001001', '9': '01010001', '0': '01011001'}


In [8]:
Braille_code_numpy = {key:np.array([int(x) for x in Braille_CODE_DICT[key]] + [0, 0])
                    for key in Braille_CODE_DICT.keys()}
subset = list(Braille_code_numpy.keys())

In [9]:
id2key = subset
key2id = {}
for idx, s in enumerate(subset):
    key2id[s] = idx

X = [ttype(Braille_code_numpy[k])for k in subset]
Y = torch.LongTensor(np.arange(0,len(X)))
print(Y.max())
print(X, Y)

tensor(42)
[tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 0., 0., 1., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 0., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 1., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 1., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 0., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 1., 0., 1., 0., 0., 0.], d

<ipython-input-9-b9b8c0fd878a>:6: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  X = [ttype(Braille_code_numpy[k])for k in subset]


In [22]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

# Three Layers

In [23]:
model = TCN(1, 43, [25, 25], kernel_size=45, dropout=0.0).cuda()
tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)


Total Weights: 86868
TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(1, 25, kernel_size=(45,), stride=(1,), padding=(44,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.0, inplace=False)
        (conv2): Conv1d(25, 25, kernel_size=(45,), stride=(1,), padding=(44,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.0, inplace=False)
        (net): Sequential(
          (0): Conv1d(1, 25, kernel_size=(45,), stride=(1,), padding=(44,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.0, inplace=False)
          (4): Conv1d(25, 25, kernel_size=(45,), stride=(1,), padding=(44,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.0, inplace=False)
        )
        (downsample): Conv1d(1, 25, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(25, 25, kern

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [24]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 5000
Trainscale = 10
device='cuda'
batch_size = 8
batches = int(np.ceil(43 / batch_size))

progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
times_100 = 0

for epoch_idx in progress_bar:
    perfs = []
    losses = []
    model.train()
    for batch_idx in range(batches):
        optimizer.zero_grad()
        loss = 0
        permute = np.arange(0, 43)
        for i in range(0, int(min(len(X) - (batch_idx*batch_size),
                              batch_size))
                       ):
            iv = X[permute[batch_idx*batch_size + i]]
            iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
            iv = iv.repeat(1,1,1,Trainscale)
            iv = iv.reshape(1,1,-1)
            tv = Y[permute[batch_idx*batch_size + i]].to(device)
            out = model(iv)
            loss += loss_func(out,
                         torch.cuda.LongTensor([tv]))
            perfs.append((torch.argmax(out, dim=-1) ==
                          tv).sum().item())


        loss = loss / min(len(X) - (batch_idx*batch_size),
                          batch_size)

        loss.backward()
        optimizer.step()


        #perfs = perfs[int(-loss_buffer_size/batch_size):]
        losses.append(loss.detach().cpu().numpy())
        #losses = losses[int(-loss_buffer_size/batch_size):]


        s = "{}:{:2} Loss: {:.4f}, Perf: {:.4f}"
        format_list = [epoch_idx, batch_idx, np.mean(losses),
                       np.sum(perfs)/((len(perfs)))]
        s = s.format(*format_list)
        progress_bar.set_description(s)
    if (np.sum(perfs)/((len(perfs))) == 1.0) & (np.mean(losses) < .11):
        times_100 += 1
        if times_100 >= 3:
            break


  0%|     | 0/5000 [00:00<?, ?it/s]     

In [25]:
model.eval()
evald = []
evaldDict = {'test_perf':[],
             'rate':[]}
for nr in [1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40]:
#for nr in range(1,20):
    perfs = []
    for batch_idx, iv in enumerate(X):
        iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
        iv = iv.repeat(1,1,1,nr)
        iv = iv.reshape(1,1,-1)
        tv = Y[batch_idx].to(device)
        out = model(iv)
        loss = loss_func(out,
                         torch.cuda.LongTensor([tv]))


        perfs.append((torch.argmax(out, dim=-1) ==
                      tv).sum().item())
        #print(torch.argmax(out, dim=-1),
        #              tv)
    evaldDict['test_perf'].append(sum(perfs)/len(perfs))
    evaldDict['rate'].append(nr)
    print(nr, sum(perfs)/len(perfs))
    evald.append({'scale':nr,
                  'perf':sum(perfs)/len(perfs)})
scale_perfs = pd.DataFrame(evald)
scale_perfs.to_pickle(join("perf", "tcn_braille_test.dill"))

1 0.023255813953488372
2 0.046511627906976744
5 0.023255813953488372
6 0.046511627906976744
7 0.046511627906976744
8 0.0
9 0.023255813953488372
10 1.0
11 0.023255813953488372
12 0.023255813953488372
13 0.0
14 0.06976744186046512
15 0.0
16 0.0
17 0.023255813953488372
18 0.0
19 0.023255813953488372
20 0.0
30 0.023255813953488372
40 0.023255813953488372
